Author: Pawel Konieczny (pkonieczny@numerix.com)

# Key/Value API

In this example we will show what are the principles behind Key/Value API.

For this example we will create an FX Barrier Pricer.

We will also introduce **NxPyTools.Create** and **NxPyTools.View** modules.

## Excel example

*Open FX_Barrier_option.xlsx*

## Python Example

### Most basic way of creating objects

First we do all necessary imports

In [ ]:
from __future__ import print_function
import NxPySDK as nx    # this is standard Numerix SDK

import datetime

First we need to create an Application object. All computations are going to be performed in that context.

In [ ]:
app = nx.Application()   # computation context
warning = nx.ApplicationWarning()     # needed to capture if something went wrong

We may look at what is in an empty Application

In [ ]:
print(app)

Then we need to start creating all objects one by one. The overall scheme is going to be the same:
  - create ApplicationCall
  - add headers and values
  - register the object in the Application

In [ ]:
# TODAY OBJECT:
# Create ApplicationCall first
c = nx.ApplicationCall()

# add headers and values
c.addValue('ID', 'TODAY')
c.addValue('NAME', 'TODAY')
c.addValue('OBJECT', 'EVENTS')
c.addValue('TYPE', 'SINGLE DATE')
c.addValue('DATES', datetime.date(2016, 7, 27))

# registering this object with Application
app.call(c, warning)

let's see if this object actually showed up in the Application

In [ ]:
print(app)

### Convenience functions

We see that this will be a repetitive task so we might as well create some convenience functions for that.

There are some convenience functions already implemented. Those are gathered in the **NxPyTools.Create module**.

In [ ]:
# import Create module:
import NxPyTools.Create as nxCreate

Now we can create objects a bit more easily. There is a number of possibilities which we encourage you to explore but here we will use the most straightforward one.

Let's start with getting some help on that module:

In [ ]:
# to find out what functions are available one may write:
help(nxCreate)

In [ ]:
# let's create USD Yield Curve
nxCreate.register_call(app=app, headers=['ID', 'OBJECT', 'TYPE', 'NOWDATE', 'CURRENCY', 'RATE/DIVIDEND'], values=['YC_USD', 'MARKET DATA', 'YIELD', 'TODAY', 'USD', 0.05])

We see that it returns registered ID for further use

In [ ]:
# let's create GBP curve:
nxCreate.register_call(app=app, headers=['ID', 'OBJECT', 'TYPE', 'NOWDATE', 'CURRENCY', 'RATE/DIVIDEND'], values=['YC_GBP', 'MARKET DATA', 'YIELD', 'TODAY', 'GBP', 0.02])

In [ ]:
# Instrument to be priced
instrument_id = nxCreate.register_call(app=app,
                       headers=['ID', 'OBJECT', 'TYPE', 'BARRIER LEVEL', 'BARRIER TYPE', 'BARRIER DIRECTION', 'CURRENCY', 'FOREIGN 1 CURRENCY', 'EXPIRY', 'PAYOUT CURRENCY'],
                       values=['FX_GBP_USD_INSTRUMENT', 'INSTRUMENT', 'FX BARRIER', 1.5, 'Knock In', 'DOWN', 'USD', 'GBP', datetime.date(2017, 7, 27), 'GBP'])

In [ ]:
print(instrument_id)

In [ ]:
# and the Pricer
nxCreate.register_call(app=app,
                      headers=['ID', 'OBJECT', 'TYPE', 'NOWDATE', 'SIGMA1', 'OPTION', 'DOMESTIC YIELD CURVE', 'FOREIGN 1 YIELD CURVE', 'SPOT PRICE', 'MODEL'],
                      values=['FX_BARRIER_PRICER', 'ANALYTIC', 'BARRIER OPTION', 'TODAY', 0.1, instrument_id, 'YC_USD', 'YC_GBP', 1.61, 'BLACK'])

In [ ]:
print(app)

### Reading results

Similarly there are *standard ways* to read results from the objects being created. From Excel one knows Object Viewer. This is achieved in the following way in SDK:

In [ ]:
d = nx.ApplicationData()   # object where to store results
app.view('FX_BARRIER_PRICER', d, warning)

Now *d* has results from the pricer. Let's see everything:

In [ ]:
print(d)

We see Object Viewer precisely. But often times we don't want to have access to everything but only to something specific.

There are different ways to achieve this but let's use some convenience functions from the **NxPyTools.View module**.

In [ ]:
import NxPyTools.View as nxView
help(nxView)

We see a range of functions we could use. Let's briefly go through some of them:

In [ ]:
# single output:
print('Delta: {}'.format(nxView.single_output(app=app, object_id='FX_BARRIER_PRICER', field_id='Delta', warning=warning)[0]))   # this function always returns list, so should access the first element

Now this is fairly convenient but if there are **more outputs** you may want to grab all of them at once:

In [ ]:
res = nxView.outputs_as_dictionary(app=app, object_id='FX_BARRIER_PRICER')  # this returns a dictionary
print(res['Delta'])   # accessing single entries in the dictionary
print(res['Gamma'])

### Writing results to an XML file

If one wants to save that structure in an XML file he can use the built in functionality:

In [ ]:
app.writeXML('C:/Python/XMLFiles/fx_pricer_manual.xml', warning)    

We will use that file in our next example